<a href="https://colab.research.google.com/github/ztach/zaliczki/blob/main/zaliczki10_22sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:,.2f}'.format
# pd.set_option('display.max_rows',None)
# pd.set_option('display.max_columns',None)

In [ ]:
pth='/content/drive/MyDrive/Colab Notebooks/'
fil = 'zaliczki2010-2022.xlsx'
url= pth + fil
data_all = ['PrzedsiębiorstwoId','Zarządca','Indeks','Wspólnota','rok(Data Płatności)','miesiąc(Data Płatności)','Suma (Remont)','Suma (Ekspl.)','Suma (Woda)','Suma (C.O.)','Suma (Śmieci)','Suma (Eks_media)','Suma (Razem)','czyUżytkowy','Ile lok u zarządcy','Ile lok u repr','IleWspólnotNaDziś','Funkcja']
data_dell = ['id','sfkprzedsiebiorstwoid','zarzadca','reprezentant','indeks','wspolnota','DataKsiegowania','eksm_media','razem','lokUz','dataWprowadzenia','lokale_u_zarz','lokale_u_repr','KtoKsiegowal','ile','fukcja','sfkdowodId']

class GetExcel:
    def __init__(self,url):
        self.url = url
        
    def get_all_df(self):
        return pd.read_excel(url)

    def get_df(self,args):
        return (pd
                .read_excel(url)
                .drop(columns=args)
        )

In [ ]:
df = GetExcel(url)
dz = df.get_df(data_dell)

In [ ]:
dz['rok'] = pd.DatetimeIndex(dz['DataPlatnosci']).year 
dh = dz.drop(columns=['DataPlatnosci'])
su = dh.query('rok > 2010').groupby('rok').sum()
su

In [ ]:
rys1 = su.groupby(['rok']).agg(
    # mc_count=('mc','count'),
    rem_sum=('remont','sum'),
    ekspl_sum=('ekspl','sum'),
    woda_sum=('woda','sum'),
    smieci_sum=('smieci','sum'),
    co_sum=('co','sum'),
    co_cor=('co','mean')
)

data = rys1[['rem_sum','ekspl_sum','woda_sum','smieci_sum','co_sum']]
skala = 1.5
lines = data.plot.line(figsize=(12*skala,9*skala))


Robię bazę danych

In [ ]:
from sqlalchemy import create_engine
pth='/content/drive/MyDrive/Colab Notebooks/'
    
engine = create_engine('sqlite:////'+pth+'test.db')

ładuję do bazy tabele z zaliczkami:

1.   'zaliczki_suma' - są to zaliczki zgrupowane wg. roku i zsumowane
2.   'zaliczki'      - tu są wszystkie zaliczki 
3.   'zaliczki2011'  - tu mamy zaliczki od roku 2011 do 2022

In [ ]:
# su.to_sql('zaliczki_suma',con=engine)

In [ ]:
data_list = engine.execute("SELECT * FROM zaliczki_suma").fetchall()
# [print(dl) for dl in data_list]

In [ ]:
# dh.to_sql('zaliczki',con=engine)

In [ ]:
# dh.query('rok > 2010').to_sql('zaliczki2011',con=engine)



---



In [ ]:
# from sqlalchemy.sql import text

# with engine.connect() as conn:
#     with conn.execution_options(yield_per=100).execute(
#         text("select * from zaliczki2011")
#     ) as result:
#         for partition in result.partitions():
#             # partition is an iterable that will be at most 100 items
#             for row in partition:
#                 print(f"{row}")



praca w SQL z utworzonymi powyżej tabelami w bazie sqlite

wynik pobranych danych wrzucany do DataFrame i robiony jest wykres

In [ ]:
result = engine.execute('SELECT rok,sum(remont) as remont, sum(ekspl) as ekspl, sum(woda) as woda, sum(co) as co, sum(smieci) as smieci FROM zaliczki2011 GROUP BY rok;')
# [print(dl) for dl in result]
dfr = pd.DataFrame(result)
result.close()

In [ ]:
from sqlalchemy.sql.expression import column
dfr.columns=['rok','remont','ekspl','woda','co', 'smieci']


In [ ]:
skala = 1.01
data = dfr[['remont','ekspl','woda','co', 'smieci']]
# lines = data.plot.line(figsize=(12*skala,9*skala))


In [ ]:
plt.plot(dfr[['rok']],data)
plt.title('zaliczki 2011-2022')
plt.xlabel('rok')
plt.ylabel('kasa')

plt.show()

In [ ]:
# help(plt.plot)